<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/BMLP_Derfansyah_Guswiranata_Kurnia_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#Perpustakaan untuk pengelolaan data
import numpy as np
import pandas as pd
#Grafik
import matplotlib.pyplot as plt #grafik dasar
import seaborn as sns
import datetime

In [190]:
#pelabelan untuk 'onehotencoder'
from sklearn.preprocessing import LabelBinarizer as lbin

In [135]:
#Bagian svm(p1)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [47]:
#Bagian klasifikasi 'randomforest'
from sklearn.ensemble import RandomForestClassifier

In [10]:
#Pengunduhan dataset ke google colab
from google.colab import files
#ke proses 'upload'

In [11]:
#proses 'upload'
uploaded = files.upload()

Saving hslclstr.csv to hslclstr.csv


In [8]:
#pengelolaan file
import os #untuk os
import time #waktu
import datetime #tanggal
import gc #pengunduhan file per batch

In [9]:
#Penyimpanan hasil sementara di google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [139]:
#inisiasi berat sampel
from sklearn.utils.class_weight import compute_sample_weight as csw #inspeksi berat dl

In [69]:
#Bagian multiprocessing melalui instalasi pool
import warnings #peringatan
import multiprocessing as mp #untuk multiprocessing
from multiprocessing import Pool

In [14]:
#penyimpanan hasil melalui dill
! pip install dill
import dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.0 MB/s eta 0:00:00


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [15]:
#Cek data
df = pd.read_csv('/content/hslclstr.csv')
dfc = df.copy() #jangan pakai data asli

In [16]:
#cek hasil
smpl = dfc.head(7)
inf = dfc.info() #informasi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   img                          64016 non-null  object 
 1   title                        64016 non-null  object 
 2   console                      64016 non-null  object 
 3   genre                        64016 non-null  object 
 4   publisher                    64016 non-null  object 
 5   total_sales                  64016 non-null  float64
 6   na_sales                     64016 non-null  float64
 7   jp_sales                     64016 non-null  float64
 8   pal_sales                    64016 non-null  float64
 9   other_sales                  64016 non-null  float64
 10  critic_score                 64016 non-null  float64
 11  release_date                 64016 non-null  object 
 12  last_update                  64016 non-null  object 
 13  kemungkinan gene

In [18]:
#inspeksi label untuk sampel
lbl = dfc['Label'].copy()
print(lbl.head(7))

0    game kurang berkenan dengan fokus pencarian ke...
1    game kurang berkenan dengan fokus pencarian ke...
2    game kurang berkenan dengan fokus pencarian ke...
3                game dengan tujuan utama pencari duit
4    game kurang berkenan dengan fokus pencarian ke...
5                game dengan tujuan utama pencari duit
6                game dengan tujuan utama pencari duit
Name: Label, dtype: object


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [20]:
#Inisiasi pembagian data
inspk = dfc.iloc[:,5:11].copy() #numerikal untuk klasifikasi kluster
inspk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_sales   64016 non-null  float64
 1   na_sales      64016 non-null  float64
 2   jp_sales      64016 non-null  float64
 3   pal_sales     64016 non-null  float64
 4   other_sales   64016 non-null  float64
 5   critic_score  64016 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB


In [21]:
#numerisasi label (identifikasi jenis label)
jnslbl = dfc['Label'].copy()
jnslbl.info()

<class 'pandas.core.series.Series'>
RangeIndex: 64016 entries, 0 to 64015
Series name: Label
Non-Null Count  Dtype 
--------------  ----- 
64016 non-null  object
dtypes: object(1)
memory usage: 500.2+ KB


In [26]:
#Sumbu x
xcls = inspk.copy() #pakai numerik

Sumbu x  **(xcls)** adalah nilai numerikal dari tabel yang terdiri dari
* Penjualan dari lima indikator (total, spesifika amerika utara, jepang, PAL, lain-lain)
* Nilai kritik

In [189]:
#Persiapan pembagian data (untuk kluster)
ycls = jnslbl #masih kata-kata
#pelabelan dengan 'ohen.onehotencoder' (pembentukan ulang)
bntkbaru, label = pd.factorize(ycls)
#inspeksi hasil
print(bntkbaru)
print(label)

[0 0 0 ... 2 2 2]
Index(['game kurang berkenan dengan fokus pencarian keuntungan',
       'game dengan tujuan utama pencari duit',
       'game favortit dengan keuntungan rendah'],
      dtype='object')


sampel kolom pelabelan

In [191]:
#pembentukan ulang
enkoding = bntkbaru.reshape(-1,1)
#inisiasi ohen
encoder = lbin()
#penyesuaian data
lblohen = encoder.fit_transform(enkoding)

In [193]:
#konversi ke df untuk kluster jadi
dflbl = pd.DataFrame(lblohen, columns=encoder.classes_)
#cek hasil
smpllbl = dflbl.head(7)
print(smpllbl) #inspeksi hasil

   0  1  2
0  1  0  0
1  1  0  0
2  1  0  0
3  0  1  0
4  1  0  0
5  0  1  0
6  0  1  0


Nilai ini **(ycls)** menjadi komponen untuk penyatuan

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [194]:
#proses scaling untuk sumbu X
rbsscal = RobustScaler() #untuk SVM
xclscl = rbsscal.fit_transform(xcls) #untuk metode SVM
#pengubahan ke data frame untuk evaluasi
dfxcl = pd.DataFrame(xclscl, columns=xcls.columns)

In [195]:
#inspeksi kolom
print(dfxcl.head(7))

   total_sales  na_sales  jp_sales  pal_sales  other_sales  critic_score
0       1016.0     637.0      99.0       9.85         3.12      1.105263
1        969.5     606.0      60.0       9.71         3.02      1.263158
2        807.5     841.0      47.0       5.49         1.78      1.210526
3        793.0     906.0       6.0       5.33         1.42      1.210526
4        754.5     618.0      41.0       6.05         2.44      0.421053
5        741.0     907.0      13.0       4.29         1.33      0.736842
6        737.0     976.0      11.0       3.73         1.14      0.789474


7 kolom

In [198]:
#Persiapan df ternormalisasi untuk Random forest classifier
mnmxscal = MinMaxScaler() #untuk RFC('RandomforestClasTree')
xclsmm = mnmxscal.fit_transform(xcls)

In [199]:
#Bentuk nilai data untuk teknik RCT
dfxclmm = pd.DataFrame(xclsmm, columns=xcls.columns)
#cek hasil
print(dfxclmm.head(7))

   total_sales  na_sales  jp_sales  pal_sales  other_sales  critic_score
0     1.000000  0.652664  0.464789   1.000000     1.000000      0.933333
1     0.954232  0.620902  0.281690   0.985787     0.967949      0.966667
2     0.794783  0.861680  0.220657   0.557360     0.570513      0.955556
3     0.780512  0.928279  0.028169   0.541117     0.455128      0.955556
4     0.742618  0.633197  0.192488   0.614213     0.782051      0.788889
5     0.729331  0.929303  0.061033   0.435533     0.426282      0.855556
6     0.725394  1.000000  0.051643   0.378680     0.365385      0.866667


In [200]:
#deklarasi nilai X dan y
X_train, X_test, y_train, y_test = train_test_split(xclscl, ycls, test_size=0.25, random_state=42) #untuk cek kondisi 1
X_train2, X_test2, y_train2, y_test2 = train_test_split(xclscl, ycls, test_size=0.225, random_state=42) #untuk RCT

In [201]:
#Inisiasi 'berat sampel'
brtsampel = csw(class_weight='balanced', y=y_test) #dipakai di kedua metode

Rincian Percobaan
* Percobaan pertama untuk kedua metode berlangsung dengan rasio (33/7)
* Percobaan kedua dengan rasio (3/1)
* Percobaan ketiga dengan rasio (31/9)

In [202]:
#Fungsi penentu akurasi melalui KNN
def train_and_evaluate(k, sample_weight=None):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Pembulatan metrik
    accuracy = accuracy_score(y_test, y_pred, sample_weight=sample_weight[:len(y_test)]) # Akurasi yang dinormalkan
    precision = precision_score(y_test, y_pred, average='weighted', sample_weight=sample_weight[:len(y_test)]) # Penyesuaian di presisi
    recall = recall_score(y_test, y_pred, average='weighted',sample_weight=sample_weight[:len(y_test)]) #Penyesuaian di kondisi recall
    f1 = f1_score(y_test, y_pred, average='weighted',sample_weight=sample_weight[:len(y_test)]) #Penyesuaian di skor F1

    # Print metrics
    print(f"Total Tetangga k={k}")
    print(f"Nilai Akurasi: {accuracy:.4f}, Presisi: {precision:.4f}, Aspek Recall: {recall:.4f}, Skor F1: {f1:.4f}")

    # Classification report
    report = classification_report(y_test, y_pred)

    if (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy >= 0.92 and precision >= 0.92 and recall >= 0.92 and f1 >= 0.92):
        return (k, accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
         (accuracy < 0.92 and precision < 0.92 and recall < 0.92 and f1 < 0.92):
        print("Hasil masih bisa disimpan.")
        return (k, accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall < 0.87 and f1 < 0.87) or \
         (accuracy > 0.92 and precision > 0.92 and recall < 0.92 and f1 < 0.92):
         print("Ada masalah di recall dan nilai f1.")
         return None
    else:
        print("Hasil tidak pantas disimpan.")
        return None
  #ke pembentukan matriks

In [203]:
#Fungsi evaluasi matriks
def generate_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm_rounded = np.round(cm.astype(float), decimals=4)  # Pembulatan untuk kemudahan nilai
    print("Hasil Confusion Matriks:")
    print(cm_rounded)
    return cm_rounded
#ke eksekusi asli

In [204]:
#Deklarasi parameter batas di KNN
param_grid = range(131, 140)
#Paralelisasi melalui KNN
if __name__ == '__main__':
    with Pool() as pool:
        results = pool.starmap(train_and_evaluate, [(k, brtsampel) for k in param_grid])

    # Filter out empty results
    filtered_results = [result for result in results if result is not None]

    # Get the best result
    best_result = max(filtered_results, key=lambda x: x[1], default=None)

    if best_result:
        # Pembulatans
        rounded_best_result = (
            best_result[0],  # Nilai awal
            round(best_result[1], 3),  # Pembulatan 1
            round(best_result[2], 3),  # Pembulatan 2
            round(best_result[3], 3),  # Pembulatan 3
            round(best_result[4], 3),  # Pembulatan 4
            best_result[5],  # Predictions (y_pred)
            best_result[6]   # Informasi tambahan
        )

        # Simpan nilai terbaik dari dill
        with open('best_knn_result.dill', 'wb') as f:
            dill.dump(rounded_best_result, f)

        _, _, _, _, _, y_pred, _ = rounded_best_result
        cm = generate_confusion_matrix(y_test, y_pred)

        # Simpan hasil matriks terbaik saja
        with open('best_confusion_matrix.dill', 'wb') as cm_file:
            dill.dump(cm, cm_file)

        # Print the best rounded result
        print("Best Result:", rounded_best_result)
#eksekusi KNN

Total Tetangga k=133
Nilai Akurasi: 0.4273, Presisi: 0.4339, Aspek Recall: 0.4273, Skor F1: 0.4064
Total Tetangga k=131
Nilai Akurasi: 0.4257, Presisi: 0.4322, Aspek Recall: 0.4257, Skor F1: 0.4036
Total Tetangga k=135
Nilai Akurasi: 0.4263, Presisi: 0.4326, Aspek Recall: 0.4263, Skor F1: 0.4048
Total Tetangga k=137
Nilai Akurasi: 0.4243, Presisi: 0.4284, Aspek Recall: 0.4243, Skor F1: 0.4033
Total Tetangga k=139
Nilai Akurasi: 0.4303, Presisi: 0.4332, Aspek Recall: 0.4303, Skor F1: 0.4132


ValueError: Found input variables with inconsistent numbers of samples: [14404, 16004]

In [174]:
#Tahap pertama RFC (pelacakan nilai ccp alpha)
nialpf = np.arange(1.1e-4, 0.11, 1.1e-4) #Nilai alfa
itmaks = 300
tolerance = 1.5e-3
best_alpha = None
best_accuracy = -np.inf
best_f1 = -np.inf
previous_best_accuracy = None
#ke iterasi pelacakan nilai ccp alpha

In [ ]:
#Fungsi inspeksi nilai ccp alpha optimal
def evaluasi_alfa(alpha):
    rfc = RandomForestClassifier(random_state=50, ccp_alpha=alpha)
    rfc.fit(X_train, y_train, sample_weight=brtsampel)
    y_pred = rfc.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    # Pembulatan akurasi
    accuracy_rounded = round(accuracy, 3)

    #penyetelan
    brtsmplnorm = brtsampel[:len(y_test)]

    # Penghindaran Peringatan di normalisasi
    f1 = f1_score(y_test, y_pred, average='weighted', sample_weight=brtsmplnorm, zero_division=0)  # Nilai f1
    precision = precision_score(y_test, y_pred, average='weighted', sample_weight=brtsmplnorm, zero_division=0)  # indikator presisi
    recall = recall_score(y_test, y_pred, average='weighted', sample_weight=brtsmplnorm, zero_division=0)  # Nilai Recall

    return alpha, accuracy_rounded, f1, precision, recall  # Return values

# Iterasi melalui jangkauan nilai alfa
for iteration in range(itmaks):
    with Pool() as pool:
        results = pool.map(evaluasi_alfa, nialpf)

    for alpha, accuracy, f1, precision, recall in results:
        if accuracy > best_accuracy or (accuracy == best_accuracy and f1 > best_f1):
            best_alpha = alpha
            best_accuracy = accuracy
            best_f1 = f1

        # Pemeriksaan untuk nilai minimal (atau optimal)
        if (accuracy >= 0.92 and precision >= 0.92 and
            recall >= 0.92 and f1 >= 0.92) or \
           (accuracy >= 0.87 and precision >= 0.87 and
            recall >= 0.87 and f1 >= 0.87):
            print(f"Optimal metrics reached with ccp_alpha: {alpha}, "
                  f"Akurasi: {accuracy:.3f}, F1 Score: {f1:.2f}, "
                  f"Precision: {precision:.2f}, Recall: {recall:.2f}. Stopping search.")
            break

    # Pengaturan Nilai alfa
    nialpf = np.linspace(max(0.001, best_alpha - 0.005), min(0.1, best_alpha + 0.005), num=10)

    # Check for tolerance to stop early
    if iteration > 0 and abs(best_accuracy - previous_best_accuracy) < tolerance:
        print("Penghentian terjadi karena sudah melewati batas toleransi.")
        break

    previous_best_accuracy = best_accuracy

# Final results output
print(f'Nilai ccp_alpha terbaik: {best_alpha}, Accuracy: {best_accuracy:.3f}, F1 Score: {best_f1:.2f}')
#ke fungsi RFC

In [ ]:
#Fungsi dengan teknik RFC
final_model = RandomForestClassifier(random_state=50, ccp_alpha=best_alpha)
final_model.fit(X_train, y_train)
y_pred_final = final_model.predict(X_test)
#Ke evaluasi

In [ ]:
#evaluasi melalui laporan dan 'confusion matrix'
conf_matrix = confusion_matrix(y_test, y_pred_final)
class_report = classification_report(y_test, y_pred_final)
#pencetakan nilai matriks
print("Confusion Matrix:")
print(conf_matrix)
print("\nLaporan Klasifikasi:")
print(class_report)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.
### Rincian pemakaian algoritma KNN (K-nearest Neighboorhood)
1. Persiapan numerisasi label **'onehotencoding'**
2. Proses standardisasi kolom numerik dengan **"Robustscaler"**
3. Eksekusi dengan teknik KNN melalui beberapa tahap :
#### 3.a. Proses Pelatihan dan evaluasi nilai gama dalam jangkauan dinamis
#### 3.b. Proses Paralelisasi untuk eksekusi dan evaluasi
#### 3.c. Tahap pencetakan Nilai akurasi, presisi 'recall'
### Rincian pemkaian algoritma RFC (Random Forest Classifier)
* Persiapan numerisasi label dengan **'onehotencoding'**
* proses standardisasi dengan metode sama (agar lebih adil)
* Eksekusi fungsi yang terdiri dari
** a. Pelatihan RFC dengan RandomSearch CV untuk modifikasi nanti
** b. Tahapan paralelisasi untuk RFC
** c. Tahap pencetakan nilai RFC

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## Evaluasi algoritma antara Hasil 'tetangga terdekat' dengan teknik 'randomforest'
### Evaluasi dengan teknik KNN
#### a. Hasil mengalmi fluktuasi rendah nilai akurasi presisi indeks 'recall' dan skor f1
#### b.
### Evaluasi dnegan teknik RFC
#### 1. Nilai masih terlalu rendah baik tanpa pemberatan ataupun pembera

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

## Modifikasi di teknik KNN

In [ ]:
#Inisiasi grid search di SVM


In [ ]:
#Lanjutan di SVM


## Modifikasi RCT

In [ ]:
#Tahap awal di RCT


In [ ]:
#Lanjutan di RCT

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Inspeksi di metode Support Vector Machine


In [ ]:
#Inspeksi di metode RandomForestClasifier


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.